this page fot creating DB and temprary jobs

Creating inatial DB

In [2]:
import sqlite3

# Connect to (or create) the SQLite database
conn = sqlite3.connect('opex.db')
cursor = conn.cursor()

# Enable foreign key support
cursor.execute("PRAGMA foreign_keys = ON;")

cursor.execute("""
CREATE TABLE IF NOT EXISTS test (
    id INTEGER PRIMARY KEY,
    name_en TEXT,
    directorate TEXT,
    catogoery TEXT

);
""")

# Table: departments
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    id INTEGER PRIMARY KEY,
    name_en TEXT,
    directorate TEXT,
    catogoery TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: accounts
cursor.execute("""
CREATE TABLE IF NOT EXISTS accounts (
    id INTEGER PRIMARY KEY,
    name_en TEXT,
    account INTEGER,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: status_master
cursor.execute("""
CREATE TABLE IF NOT EXISTS status_master (
    id INTEGER PRIMARY KEY,
    name_en TEXT,
    category TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: po
cursor.execute("""
CREATE TABLE IF NOT EXISTS po (
    id INTEGER PRIMARY KEY,
    amount INTEGER,
    po_number INTEGER,
    po_date DATE,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: services
cursor.execute("""
CREATE TABLE IF NOT EXISTS services (
    id INTEGER PRIMARY KEY,
    description TEXT,
    vendor_name TEXT,
    start_date DATE,
    end_date DATE,
    email TEXT,
    mobile INTEGER,
    remarks TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: mrn
cursor.execute("""
CREATE TABLE IF NOT EXISTS mrn (
    id INTEGER PRIMARY KEY,
    name_en TEXT,
    completion INTEGER,
    invoice_no INTEGER,
    unit_mrn INTEGER,
    mrn_amount INTEGER,
    mrn_balance INTEGER,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: amc_contracts
cursor.execute("""
CREATE TABLE IF NOT EXISTS amc_contracts (
    id INTEGER PRIMARY KEY,
    domain TEXT,
    c_code TEXT,
    expense_type TEXT,
    cost_center TEXT,
    account_code TEXT,
    account_name TEXT,
    line_budget TEXT,
    vendor TEXT,
    email TEXT,
    mobile INTEGER,
    sub_category TEXT,
    ifrs_16 TEXT,
    start_date DATE,
    end_date DATE,
    type_of_cost TEXT,
    type_of_amc TEXT,
    remarks TEXT,
    cvd_status TEXT,
    risk_comment TEXT,
    procurement_comment TEXT,
    quotation_received BOOLEAN,
    created TIMESTAMP,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER
);
""")

# Table: amc_budgets
cursor.execute("""
CREATE TABLE IF NOT EXISTS amc_budgets (
    id INTEGER PRIMARY KEY,
    ref_amc_contract INTEGER UNIQUE,
    year INTEGER,
    units INTEGER,
    unit_cost REAL,
    total_budget REAL,
    review_required BOOLEAN DEFAULT 0,
    approved_budget REAL,
    comment TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER,
    FOREIGN KEY (ref_amc_contract) REFERENCES amc_contracts(id)
);
""")

# Table: amc_pos
cursor.execute("""
CREATE TABLE IF NOT EXISTS amc_pos (
    id INTEGER PRIMARY KEY,
    ref_amc_contract INTEGER,
    year INTEGER,
    po_amount REAL,
    po_number TEXT,
    pr_amount REAL,
    pr_number TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER,
    FOREIGN KEY (ref_amc_contract) REFERENCES amc_contracts(id)
);
""")

# Table: amc_status
cursor.execute("""
CREATE TABLE IF NOT EXISTS amc_status (
    id INTEGER PRIMARY KEY,
    ref_amc_contract INTEGER,
    status TEXT,
    updated_by TEXT,
    updated_on TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER,
    FOREIGN KEY (ref_amc_contract) REFERENCES amc_contracts(id)
);
""")

# Table: pr
cursor.execute("""
CREATE TABLE IF NOT EXISTS pr (
    id INTEGER PRIMARY KEY,
    pr_number INTEGER,
    approval_amount INTEGER,
    approved BOOLEAN,
    contract_reference TEXT,
    ref_budget_details INTEGER,
    ref_emp INTEGER,
    ref_departments INTEGER,
    ref_accounts INTEGER,
    ref_mrn INTEGER,
    ref_po INTEGER,
    ref_services INTEGER,
    ref_status_master INTEGER,
    comments TEXT,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER,
    FOREIGN KEY (ref_accounts) REFERENCES accounts(id),
    FOREIGN KEY (ref_budget_details) REFERENCES amc_budgets(id),
    FOREIGN KEY (ref_departments) REFERENCES departments(id),
    FOREIGN KEY (ref_mrn) REFERENCES mrn(id),
    FOREIGN KEY (ref_po) REFERENCES po(id),
    FOREIGN KEY (ref_services) REFERENCES services(id),
    FOREIGN KEY (ref_status_master) REFERENCES status_master(id)
);
""")

# Commit changes and close connection
conn.commit()
conn.close()


testign negaitve figures 

In [8]:
import sqlite3

conn = sqlite3.connect("opex.db")
cur = conn.cursor()

# Create table with constraint
cur.execute("""
CREATE TABLE IF NOT EXISTS pr (
    id INTEGER PRIMARY KEY,
    approval_amount INTEGER NOT NULL CHECK (approval_amount > 0)
)
""")

# Test inserting valid and invalid values
try:
    cur.execute("INSERT INTO pr (approval_amount) VALUES (?)", (100,))  # Valid
    cur.execute("INSERT INTO pr (approval_amount) VALUES (?)", (-10,))  # Invalid
except sqlite3.IntegrityError as e:
    print("Constraint failed:", e)

conn.commit()
conn.close()


update DB remove pr, po and services tables 

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("opex.db")
cursor = conn.cursor()

# Enable foreign key support
cursor.execute("PRAGMA foreign_keys = ON;")

# Recreate the updated schema for the listed tables
cursor.executescript("""


DROP TABLE IF EXISTS amc_contracts;
CREATE TABLE amc_contracts (
    id INTEGER PRIMARY KEY,
    pr_number INTEGER,
    domain TEXT,
    c_code TEXT,
    expense_type TEXT,
    cost_center TEXT,
    approval_amount INTEGER,
    approved BOOLEAN,
    contract_reference TEXT,
    line_budget TEXT,
    vendor TEXT,
    sub_category TEXT,
    ifrs_16 TEXT,
    email TEXT,
    mobile INTEGER,
    start_date DATE,
    end_date DATE,
    year integer, 
    type_of_cost TEXT,
    type_of_amc TEXT,
    remarks TEXT,
    cvd_status TEXT,
    risk_comment TEXT,
    procurement_comment TEXT,
    quotation_received BOOLEAN,
    ref_departments INTEGER,
    ref_account INTEGER,
    ref_status INTEGER,
    created TIMESTAMP,
    created_by TEXT,
    modified TIMESTAMP,
    modified_by TEXT,
    version INTEGER,
    FOREIGN KEY (ref_departments) REFERENCES departments(id),
    FOREIGN KEY (ref_account) REFERENCES accounts(id),
    FOREIGN KEY (ref_status) REFERENCES status_master(id)
);

""")

conn.commit()
conn.close()


import deoartments data 

In [3]:
import sqlite3
import pandas as pd
from datetime import datetime

# Load the CSV file (excluding 'id' if present)
csv_path = r"C:\Users\PREDATOR\OneDrive\Desktop\departments.csv"
df = pd.read_csv(csv_path)

# Optional: drop 'id' column if it exists
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Fill metadata columns
df['created'] = datetime.now()
df['created_by'] = 'admin'
df['modified'] = datetime.now()
df['modified_by'] = 'admin'
df['version'] = 1

# Reorder columns to match table schema (excluding 'id')
columns = [
    'name_en', 'directorate', 'catogoery',
    'created', 'created_by', 'modified', 'modified_by', 'version'
]

# Connect to the database and insert data
conn = sqlite3.connect("opex.db")
df[columns].to_sql('departments', conn, if_exists='append', index=False)
conn.close()

print("✅ Data imported into departments table.")


✅ Data imported into departments table.


import accounts data 

In [5]:
import sqlite3
import pandas as pd
from datetime import datetime

# Load Excel file
excel_path = r"C:\Users\PREDATOR\OneDrive\Desktop\accounts.xlsx"
df = pd.read_excel(excel_path)

# Drop 'id' column if it exists (since it's auto-incremented in the DB)
if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)

# Fill metadata columns
df['created'] = datetime.now()
df['created_by'] = 'admin'
df['modified'] = datetime.now()
df['modified_by'] = 'admin'
df['version'] = 1

# Reorder columns to match the DB table structure (excluding 'id')
columns = [
    'name_en', 'account',
    'created', 'created_by', 'modified', 'modified_by', 'version'
]

# Connect to SQLite and insert data
conn = sqlite3.connect("opex.db")
df[columns].to_sql('accounts', conn, if_exists='append', index=False)
conn.close()

print("✅ Data successfully imported into accounts table.")


✅ Data successfully imported into accounts table.
